In [0]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import spacy


In [0]:
TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train, test = datasets.IMDB.splits(TEXT, LABEL)

train, valid = train.split()

In [14]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 399213/400000 [00:22<00:00, 18252.83it/s]

In [0]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64, 
    device = device)

In [0]:
class CNN(nn.Module):
    def __init__(self, v_size, emb_dim, filter_count, f_size, out_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(v_size, emb_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = emb_dim, 
                                              out_channels = filter_count, 
                                              kernel_size = fs)
                                    for fs in f_size
                                    ])
        
        self.fc = nn.Linear(len(f_size) * filter_count, out_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        
        embedded = self.embedding(text)
        
        embedded = embedded.permute(0, 2, 1)
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)
            

In [104]:
in_dim = len(TEXT.vocab)
emb_dim = 100
filter_count = 100
f_size = [3,4,5]
out_dim = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(in_dim, emb_dim, filter_count, f_size, out_dim, DROPOUT, PAD_IDX)

print(model)

CNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [105]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,620,801 trainable parameters


In [106]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.5723, -0.8483, -0.0487,  ..., -0.0872, -0.9338,  0.9880],
        [-0.5999, -0.8135, -0.4170,  ..., -0.2763, -0.7891, -0.4087],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.5787,  1.5400, -0.3264,  ..., -0.4411,  0.1478, -0.5359],
        [-0.0259,  0.2626, -0.4150,  ...,  0.2496,  1.0473, -0.8566],
        [-0.5618,  0.5658, -0.2214,  ..., -0.2283,  0.4419,  0.5798]])

In [0]:
model.embedding.weight.data[TEXT.vocab.stoi[TEXT.unk_token]] = torch.zeros(emb_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(emb_dim)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(predictions, y):
    rounded = torch.round(torch.sigmoid(predictions))
    correct = (rounded == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def Train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        accuracy = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            accuracy = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [113]:
global_loss = float('inf')

for epoch in range(5):

    start_time = time.time()
    
    train_loss, train_accuracy = Train(model, train_iterator, optimizer, criterion)
    local_loss, valid_accuracy = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if local_loss < global_loss:
        global_loss = local_loss
        torch.save(model.state_dict(), 'ForthModel.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_accuracy*100:.2f}%')
    print(f'\t Val. Loss: {local_loss:.3f} |  Val. Acc: {valid_accuracy*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.656 | Train Acc: 61.09%
	 Val. Loss: 0.521 |  Val. Acc: 77.16%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.435 | Train Acc: 80.01%
	 Val. Loss: 0.359 |  Val. Acc: 85.10%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.312 | Train Acc: 86.86%
	 Val. Loss: 0.339 |  Val. Acc: 85.55%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.227 | Train Acc: 90.95%
	 Val. Loss: 0.312 |  Val. Acc: 86.97%
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.163 | Train Acc: 93.91%
	 Val. Loss: 0.325 |  Val. Acc: 87.20%


In [114]:
model.load_state_dict(torch.load('ForthModel.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.334 | Test Acc: 85.65%


In [0]:
nlp = spacy.load('en')

def sentiment_prediction(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [0]:
predict_sentiment(model, "AI is terrible")


In [0]:
predict_sentiment(model, "AI is future")
